<a href="https://colab.research.google.com/github/Kambey-san/NNWs_Basic/blob/HW_lesson_05/HW_lesson_05_Task_2_clb_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Урок 5. Рекуррентные нейронные сети/ Домашнее задание

2. Попробуйте изменить параметры нейронной сети, генерирующей текст таким образом, чтобы добиться генерации как можно более осмысленного текста. Пришлите лучший текст из получившихся и опишите предпринятые для его получения действия. Можно использовать текст другого произведения

Использовалась поэма М. Ю. Лермонтова "Мцыри"

In [25]:
from google.colab import files
uploaded = files.upload()

Saving Lermontov_Mcyri.txt to Lermontov_Mcyri.txt


In [26]:
# построчное чтение из примера с текстом
with open("Lermontov_Mcyri.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("utf-8", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)

In [59]:
# Удалим неинформативные строки
text = text.replace("* * *", "")
text = text.replace("***", "")
text = text.replace("  …", "")
text = text.replace("…", "")
text[:1000]

'Михаил Юрьевич Лермонтов Мцыри Библиотека школьника – Мцыри – на грузинском языке значит «неслужащий монах», нечто вроде «послушника». (Прим. Лермонтова.) Вкушая, вкусих мало меда, и се аз умираю. 1-я Книга Царств 1 Немного лет тому назад, Там, где, сливаяся, шумят, Обнявшись, будто две сестры, Струи Арагвы и Куры, Был монастырь. Из-за горы И нынче видит пешеход Столбы обрушенных ворот, И башни, и церковный свод; Но не курится уж под ним Кадильниц благовонный дым, Не слышно пенье в поздний час Молящих иноков за нас. Теперь один старик седой, Развалин страж полуживой, Людьми и смертию забыт, Сметает пыль с могильных плит, Которых надпись говорит О славе прошлой – и о том, Как, удручен своим венцом, Такой-то царь, в такой-то год, Вручал России свой народ.  И божья благодать сошла На Грузию! Она цвела С тех пор в тени своих садов, Не опасаяся врагов, 3а гранью дружеских штыков. 2 Однажды русский генерал Из гор к Тифлису проезжал; Ребенка пленного он вез. Тот занемог, не перенес Трудов да

In [60]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers import SimpleRNN, LSTM, GRU
from keras.models import Sequential

In [61]:
import tensorflow as tf

In [62]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
tf.random.set_seed(48)

In [63]:
chars = set([c for c in text])
nb_chars = len(chars)

In [64]:
# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

In [65]:
# для удобства выберете фиксированную длину последовательность 10 символов
SEQLEN, STEP = 10, 1
input_chars, label_chars = [], []

In [66]:
# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

In [67]:
# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=bool)
y = np.zeros((len(input_chars), nb_chars), dtype=bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [75]:
# установка ряда метапараметров  для нейронной сети и процесса тренировки
BATCH_SIZE, HIDDEN_SIZE = 1128, 512 #128, 128
NUM_ITERATIONS = 5 # was changed
NUM_EPOCHS_PER_ITERATION = 10 #1
NUM_PREDS_PER_EPOCH = 200 #100

In [76]:
# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой.
За ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы в
вероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию
ссылающуются на предсказание one-hot encoding лейбл символа
'''

'\nСоздание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой.\nЗа ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы в\nвероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию\nссылающуются на предсказание one-hot encoding лейбл символа\n'

In [77]:
model = Sequential()
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=True,
        input_shape=(SEQLEN, nb_chars),
        #dropout=0.2,
        #unroll=True
    )
)

In [78]:
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=True,
        input_shape=(SEQLEN, nb_chars),
        #dropout=0.2,
        #unroll=True
    )
)

In [79]:
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        #dropout=0.2,
        #unroll=True
    )
)

In [80]:
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [81]:
# выполнение серий тренировочных и демонстрационных итераций
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()

Итерация #: 0
Epoch 1/10
18/18 [==============================] - 109s 6s/step - loss: 3.8247
Epoch 2/10
18/18 [==============================] - 103s 6s/step - loss: 3.3395
Epoch 3/10
18/18 [==============================] - 102s 6s/step - loss: 3.2897
Epoch 4/10
18/18 [==============================] - 102s 6s/step - loss: 3.2303
Epoch 5/10
18/18 [==============================] - 101s 6s/step - loss: 3.0918
Epoch 6/10
18/18 [==============================] - 103s 6s/step - loss: 2.9309
Epoch 7/10
18/18 [==============================] - 101s 6s/step - loss: 2.7784
Epoch 8/10
18/18 [==============================] - 100s 6s/step - loss: 2.5969
Epoch 9/10
18/18 [==============================] - 102s 6s/step - loss: 2.5036
Epoch 10/10
18/18 [==============================] - 101s 6s/step - loss: 2.4384
Генерация из посева: ашни, и це
ашни, и цел стой Полотой стой Постотой стой Постотой стой Постотой стой Постотой стой Постотой стой Постотой стой Постотой стой Постотой стой Постотой ст

**Upgrade:**
- увеличил длину генерируемого текста до 200 символов.
- сделал предобработку текста, удалив неинформативные символы.
- увеличил число итераций до 5, число эпох до 10, число слоёв GRU до 3-х и число нейронов в слоях до 512
- увеличил BATCH_SIZE до 1128.

**Итого:**

Субъективно самый лучший вариант коррелирует с последней генерацией (объективно  миним. loss): "аз молодой
аз молодой. И странной сладкою тоской Опять моя заныла грудь. Я долго силился вздохнуть – И пробудился. Уж луна Вверху сияла, и одна Лишь тучка кралася за ней, Как за добычею своей, Как тополь, царь ее полей! Н"
